In [1]:
# Je ne sais pas vous, mais personnellement j'ai un écran large et ça m'agace
# d'avoir le notebook juste au milieu
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

### Chargement des librairies et définition fonction coordonnées->UTM

In [2]:
# librairies mathématiques et de gestion
import pandas as pd
import numpy as np

# librairie de conversion de coordonnées géo
from convertbng.util import convert_bng, convert_lonlat

def wgs84_to_web_mercator(df, lon="LON", lat="LAT"):
    '''
    Cette fonction calcule les coordonnées
    UTM (mercator) à partir de la longitude et de la latitude
    df : le dataframe où se trouvent les données à convertir
    lon : nom de la colonne pour les longitudes
    lat : nom de la colonne pour les latitudes
    return : retourne le dataframe en y ajoutant les colonnes 
    pour les coordonnées x_utm et y_utm
    '''
    k = 6378137
    df.loc[:, "x_utm"] = df.loc[:, lon] * (k * np.pi/180.0)
    df.loc[:, "y_utm"] = np.log(np.tan((90 + df.loc[:, lat]) * np.pi/360.0)) * k
    return df

### Chargement datasets

In [3]:
# j'ai passé les data en csv, avec un nom un peu moins pénible
# la lecture doit se faire avec latin-1 pour certain caractère spéciaux
df = pd.read_csv('data/Full_dataset.csv',sep=',', encoding='latin-1')

# On charge également les coordonnées des stations
station_coord = pd.read_csv('data/Station_coord.csv')

C:\Users\Roman\anaconda3\envs\ptorch\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (0,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df.info()
station_coord.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1258146 entries, 0 to 1258145
Data columns (total 15 columns):
 #   Column                    Non-Null Count    Dtype 
---  ------                    --------------    ----- 
 0   IncidentNumber            1258146 non-null  object
 1   TimeMobilised             1258146 non-null  object
 2   TimeMobile                1258146 non-null  object
 3   TimeArrived               1258146 non-null  object
 4   TimeLeft                  1212722 non-null  object
 5   TimeReturned              1041477 non-null  object
 6   DeployedFromStation_Name  1258146 non-null  object
 7   DateOfCall                1258146 non-null  object
 8   CalYear                   1258146 non-null  int64 
 9   TimeOfCall                1258146 non-null  object
 10  HourOfCall                1258146 non-null  int64 
 11  IncidentGroup             1258146 non-null  object
 12  SpecialServiceType        291762 non-null   object
 13  Easting_rounded           1258146 non-null

On retire les noms de stations qui ne sont plus présents sur 
https://www.london-fire.gov.uk/community/your-borough/, <br/>
Car ça devient compliqué d'en extraire les coordonnées avec des services gratuits
(avec des services payants, c'est une autre histoire). <br/><br/>
Avec le print(station absente, station, data1, date2), on peut voir les dates où ces stations ont envoyé des effectifs sur les incidents, et donc suggérer que ces stations ont effectivement fermée et ne sont plus pertinentes pour l'analyse.

In [5]:
#print(df.DeployedFromStation_Name.unique())
#print(station_coord.Station.unique())
# On fait une boucle de vérification, qui affichera
# si des stations sont dans df mais pas dans station_coord
to_drop = []
nb_station = len(df.DeployedFromStation_Name.unique())
# petite variable pour contrôler le taux de réduction
calcul = 0
for sta in df.DeployedFromStation_Name.unique():
    if not np.isin(sta, station_coord.Station.unique()):
        print("station absente de station_coord :", 
              sta, 
              df[df.DeployedFromStation_Name==sta].CalYear.min(),
              df[df.DeployedFromStation_Name==sta].CalYear.max())
        df = df[df.DeployedFromStation_Name!=sta]
        calcul += 1
print('La base de données a été réduite de : ', round(100-(calcul/nb_station)*100, 2), ' %')

station absente de station_coord : Clerkenwell 2009 2014
station absente de station_coord : Southwark 2009 2014
station absente de station_coord : Belsize 2009 2014
station absente de station_coord : Woolwich 2009 2014
station absente de station_coord : Silvertown 2009 2014
station absente de station_coord : Bow 2009 2014
station absente de station_coord : Westminster 2009 2014
station absente de station_coord : Kingsland 2009 2014
station absente de station_coord : Knightsbridge 2009 2014
station absente de station_coord : Downham 2009 2014
station absente de station_coord : Langley 2009 2016
station absente de station_coord : Loughton 2009 2016
station absente de station_coord : Rickmansworth 2009 2016
station absente de station_coord : Epsom 2009 2016
station absente de station_coord : Reigate 2009 2016
station absente de station_coord : Walton 2009 2015
station absente de station_coord : Cheshunt 2009 2016
station absente de station_coord : Orsett 2009 2014
station absente de stati

# Calcul des coordonnées et estimation de la distance euclidienne entre une station de départ et le lieux de l'incident
## Etapes :
- Calculer les coordonnées des incidents. Ces coordonnées sont rapportées en "Ordnance Survey National Grid" (Easting, Northing) <br/> 
car les anglais ne font rien comme tout le monde. On doit donc déduire les longitudes et latitudes correspondantes, <br/>
puis passer le tout en coordonnées UTM (Universal Transverse Mercator)
<br/>
- Calculer directement les coordonnées UTM pour les stations (puisqu'on a récolté les latitudes et longitudes)

In [6]:
# On sélectionne IncidentGroup, Easting_rounded, Northing_rounded, IncidentStationGround
# coord = df.iloc[:, [5, 21, 22, 24]] 
# Associons les types d'incident à une couleur
def color_(x):
    Type=['Special Service','Fire','False Alarm']  
    color=['blue','red','cyan']
    return color[Type.index(x)]
df.loc[:, 'color'] = df.IncidentGroup.apply(color_) 

# les colonnes de coordonnées OSNG
eastings = df.Easting_rounded
northings = df.Northing_rounded

# Conversion en longitude, latitude
res_list_en = np.array(convert_lonlat(eastings, northings))
df.loc[:, 'd_long'] = res_list_en[0, :]
df.loc[:, 'd_lat'] = res_list_en[1, :]

# Ajout coordonnées utm
df = wgs84_to_web_mercator(df, 'd_long', 'd_lat')

# Ajout coordonnées utm
station_coord = wgs84_to_web_mercator(station_coord, 'long', 'lat')

# La librairie Tqdm permet d'utiliser un bon paquet de fonction en affichant une 
# barre de progression, ce qui évite de laisser patiner le pc dans la semoule en
# ne sachant pas si il a planté où si il est toujours dans le cyber-goulag des boucles
# for.
from tqdm import tqdm

# Notre DataFrame comporte près de 1 000 000 observations,
# Les allocations dynamiques font perdre du temps,
# alors on crée les colonnes en amont au cas où
df.loc[:, 'o_long'] = 0
df.loc[:, 'o_lat'] = 0
df.loc[:, 'o_x_utm'] = 0
df.loc[:, 'o_y_utm'] = 0
df.loc[:, 'dist_euclidian'] = 0
# Distance euclidienne
#. . . . . . . . . . O (x2, y2)
#. . . . . . . . . / .
#. . . . . . . . / . .
#. . . . . . . / . . .
#. . . . . . / . . . .
#. . . . . / . . . . .
#. . . . O . . . . . . 
#    (x1, y1)
# sqrt( (x1 - x2)**2 + (y1 - y2)**2)
def myfunc(d):
    # on fait correspondre la station de la ligne (action d'un véhicule)
    # et on récupère les coordonnées du point de départ correspondant
    lon = station_coord[station_coord.Station == d.DeployedFromStation_Name].long.values[0]
    lat = station_coord[station_coord.Station == d.DeployedFromStation_Name].lat.values[0]
    o_x = station_coord[station_coord.Station == d.DeployedFromStation_Name].x_utm.values[0]
    o_y = station_coord[station_coord.Station == d.DeployedFromStation_Name].y_utm.values[0]
    d['o_long'] = lon
    d['o_lat'] = lat
    d['o_x_utm'] = o_x
    d['o_y_utm'] = o_y
    d['dist_euclidian'] = np.sqrt( (d['x_utm'] - o_x)**2 + (d['y_utm'] - o_y)**2)
    return d

# Un librairie sympa pour les tâches longues c'est Tqdm,
# Qui permet d'afficher la progression d'une tâche
# (pratique pour voir si le kernel est mort ou encore vivant)
# Pour utiliser Tqdm avec pandas, on doit d'abord créer une instance
# qui va prendre en charge le support de la méthode apply
# on utilise alors df.progress_apply()
from tqdm.autonotebook import tqdm
tqdm.pandas()
df = df.progress_apply(myfunc, axis=1)


C:\Users\Roman\anaconda3\envs\ptorch\lib\site-packages\ipykernel_launcher.py:66: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


  0%|          | 0/1190683 [00:00<?, ?it/s]

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1190683 entries, 0 to 1258145
Data columns (total 25 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   IncidentNumber            1190683 non-null  object 
 1   TimeMobilised             1190683 non-null  object 
 2   TimeMobile                1190683 non-null  object 
 3   TimeArrived               1190683 non-null  object 
 4   TimeLeft                  1148452 non-null  object 
 5   TimeReturned              974961 non-null   object 
 6   DeployedFromStation_Name  1190683 non-null  object 
 7   DateOfCall                1190683 non-null  object 
 8   CalYear                   1190683 non-null  int64  
 9   TimeOfCall                1190683 non-null  object 
 10  HourOfCall                1190683 non-null  int64  
 11  IncidentGroup             1190683 non-null  object 
 12  SpecialServiceType        275800 non-null   object 
 13  Easting_rounded           1

In [8]:
df.to_csv(r'data/data_with_coord.csv', index = False)